monitoring gpu

In [ ]:
!nvidia-smi

dependencies

In [ ]:
!pip install opendatasets==0.1.22
!pip install diffusers
!pip install transformers==4.25.1
!pip install scipy==1.7.3
!pip install ftfy==6.1.1
!pip install "ipywidgets>=7,<8"
!pip install accelerate==0.15.0
!pip install tensorflow>=2.9

In [ ]:
pip install -q --upgrade accelerate

In [ ]:
!pip install transformers --upgrade

kills the runtime



In [ ]:
import os
os.kill(os.getpid(), 9)


importing librarires

In [ ]:
import os
import sys
import numpy as np
import re
import cv2
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
from diffusers import StableDiffusionInpaintPipeline
import PIL
from huggingface_hub import notebook_login
from google.colab import output

In [ ]:
output.enable_custom_widget_manager()

In [ ]:
INPUT_SHAPE = 128

use the huggingface token here

In [ ]:
notebook_login()

connect drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DataSynthesis

In [ ]:
import torch
import cv2
import PIL
from diffusers import *
from diffusers import utils
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import load_image, make_image_grid


def get_images(image_path, mask_path):

  original_mask = cv2.imread(mask_path)


    # Thresholding on the grayscale version of the mask image
  _, im_bw = cv2.threshold(cv2.cvtColor(original_mask, cv2.COLOR_BGR2GRAY), 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Save the thresholded image with the same filename as the mask imageL
  cv2.imwrite(mask_path, im_bw)
  fixed_mask = cv2.imread(mask_path)


    # Apply blurring to the mask image
  blur = cv2.blur(fixed_mask, (15, 15), 0)
  cv2.imwrite("tmp-blur.bmp", blur)

  blurred_image = PIL.Image.open("tmp-blur.bmp").convert("RGB").resize((512, 512))
  normal_image = PIL.Image.open(image_path).convert("RGB").resize((512, 512))

  # Initialize the Stable Diffusion pipeline
  repo_id = "stabilityai/stable-diffusion-2-inpainting"
  pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, revision="fp16")
  pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
  pipe = pipe.to("cuda")

  # Prompt(background and negative)
  #  negative prompt - a way to use Stable Diffusion in a way that allows the user to specify what he doesn't want to see, without any extra input.
  BACKGROUND_PROMPT= "Overhead satellite photograph of a densely packed urban city with rooftops,metropolitan city, roads, highway, medium-high spatial resolution, it should cover 300m,  4K, satellite image, buildings, extremely detailed, photorealistic,highlighting the intricate textures of weathered metal, sun-bleached tiles"
  BACKGROUND_NEGATIVE_PROMPT ="grid, box, plastic-like appearance, flat lighting, unnatural highlights, artificial panels, cartoonish, stylized, pixelated, smooth textures, uniform colors, lack of shadows"

  inverted_m = PIL.ImageOps.invert(PIL.Image.open(mask_path).convert("RGB").resize((512, 512)))

  no_panel_image = pipe(prompt=BACKGROUND_PROMPT, image=normal_image, mask_image=blurred_image, num_inference_steps=25).images[0]
  modified_image = pipe(prompt=BACKGROUND_PROMPT, negative_prompt=BACKGROUND_NEGATIVE_PROMPT, image=no_panel_image, mask_image=inverted_m , num_inference_steps=100).images[0]
  modified_image.paste(normal_image, mask=PIL.Image.open(mask_path).convert("RGB").resize((512, 512)).convert('L'))

  #returns modified_image and mask
  return modified_image, PIL.Image.open(mask_path).convert("RGB").resize((512, 512))

function call

In [ ]:
modd_image,orm_image=get_images("/content/drive/MyDrive/pv/PV03/PV03_Rooftop/PV03_314658_1199334.bmp","/content/drive/MyDrive/pv/PV03/PV03_Rooftop/PV03_314658_1199334_label.bmp")